In [14]:
%pip install avro kafka-python

In [15]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

In [16]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [17]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [18]:
!wget https://raw.githubusercontent.com/pvateekul/2110446_DSDE_2023s2/main/code/Week09_DataIngestion/Assignment/result.avsc
!wget https://raw.githubusercontent.com/pvateekul/2110446_DSDE_2023s2/main/code/Week09_DataIngestion/Assignment/submit.avsc
!wget https://raw.githubusercontent.com/pvateekul/2110446_DSDE_2023s2/main/code/Week09_DataIngestion/Assignment/transaction.avsc

--2024-03-25 15:46:06--  https://raw.githubusercontent.com/pvateekul/2110446_DSDE_2023s2/main/code/Week09_DataIngestion/Assignment/result.avsc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 359 [text/plain]
Saving to: ‘result.avsc.1’

result.avsc.1       100%[===================>]     359  --.-KB/s    in 0s      

2024-03-25 15:46:06 (18.1 MB/s) - ‘result.avsc.1’ saved [359/359]

--2024-03-25 15:46:06--  https://raw.githubusercontent.com/pvateekul/2110446_DSDE_2023s2/main/code/Week09_DataIngestion/Assignment/submit.avsc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sen

In [19]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [20]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'lab.aimet.tech:9092'

In [21]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [22]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [23]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [24]:
VID = 'V088882'
TOKEN = '99baded1c9c64c4380a057bf056438f7'

In [25]:
for message in txconsumer:
	txid = message.value['txid']
	payer = message.value['payer']
	payee = message.value['payee']
	amount = message.value['amount']

	signature = gen_signature(txid, payer, payee, amount, TOKEN)
	submit = {'vid': VID, 'txid': txid, 'signature': signature}
	producer.send('submit', serialize(submitschema, submit))

	verifiedTransaction = None
	for message in resultconsumer:
		if message.value['vid'] == VID and message.value['txid'] == txid and message.value['code'] == 200:
			print(f"Transaction {txid} is verified")
			verifiedTransaction = message.value
			break

	if verifiedTransaction:
		print(verifiedTransaction)
		break


Transaction TX00892 is verified
{'timestamp': 1711381578, 'vid': 'V088882', 'txid': 'TX00892', 'checksum': '8986ccd72dac6df7f584b262835a50d8', 'code': 200, 'message': 'Confirm'}
